In [162]:
TRAIN_PATH = "/bohr/train-08bw/v1/"
csv_path = TRAIN_PATH + "pendulum_train.csv"

In [163]:
import numpy as np
import torch
from torch import nn
from scipy.integrate import odeint
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

In [253]:
import numpy as np
import torch
import torch.optim as optim
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import math

class SinglePendulum(nn.Module):
    def __init__(self):
        super().__init__()
        # Initialize parameters with reasonable guesses
        self.alpha = nn.Parameter(torch.tensor([0.4], requires_grad=True))
        self.beta_1 = nn.Parameter(torch.tensor([2.0], requires_grad=True))  # g/l ≈ 9.8 for l=1
        self.beta_2 = nn.Parameter(torch.tensor([8.0], requires_grad=True))  # Slightly higher than beta_1

    def forward(self, theta, omega, add_force=False):
        # Ensure parameters are broadcast correctly
        alpha = self.alpha.view(1)
        if not add_force:
            beta = self.beta_1.view(1)
        else:
            beta = self.beta_2.view(1)
        
        # Calculate acceleration for each time step
        a = -alpha * omega - beta * torch.sin(theta)
        return a

def derivative(t, theta):
    """Calculate numerical derivative using central differences"""
    dtheta = np.zeros_like(theta)
    dtheta[1:-1] = (theta[2:] - theta[:-2]) / (t[2:] - t[:-2])
    dtheta[0] = (theta[1] - theta[0]) / (t[1] - t[0])
    dtheta[-1] = (theta[-1] - theta[-2]) / (t[-1] - t[-2])
    return dtheta

def load_data(csv_path):
    """Load data from CSV file"""
    ttheta = pd.read_csv(csv_path)
    t = np.array(ttheta["t"])
    theta = np.array(ttheta["theta"])
    return t, theta

def split_data(t, theta, threshold=1):
    """Split data into pre-force and post-force segments"""
    theta_1, t_1 = [], []
    theta_2, t_2 = [], []
    force_flag = False
    
    for i in range(len(t)):
        if i == 0:
            t_1.append(t[i])
            theta_1.append(theta[i])
            continue
            
        if t[i] - t[i-1] < threshold:
            if not force_flag:
                t_1.append(t[i])
                theta_1.append(theta[i])
            else:
                t_2.append(t[i])
                theta_2.append(theta[i])
        else:
            force_flag = True
            t_2.append(t[i])
            theta_2.append(theta[i])
    
    return np.array(t_1), np.array(theta_1), np.array(t_2), np.array(theta_2)


In [254]:
def load_data(csv_path):
    """Load data from CSV file"""
    ttheta = pd.read_csv(csv_path)
    t = np.array(ttheta["t"])
    theta = np.array(ttheta["theta"])



    
    return t, theta

In [297]:
def simulate_trajectory(model, t_points, theta_init, omega_init, apply_force):
    """Simulate pendulum trajectory using current model parameters"""
    theta_sim = np.zeros_like(t_points)
    omega_sim = np.zeros_like(t_points)
    
    theta_sim[0] = theta_init
    omega_sim[0] = omega_init
    
    alpha = model.alpha.item()
    beta_1 = model.beta_1.item()
    beta_2 = model.beta_2.item()
    
    for i in range(1, len(t_points)):
        dt = t_points[i] - t_points[i-1]
        beta = beta_2 if apply_force else beta_1
        acceleration = -alpha * omega_sim[i-1] - beta * math.sin(theta_sim[i-1])
        omega_sim[i] = omega_sim[i-1] + acceleration * dt
        theta_sim[i] = theta_sim[i-1] + omega_sim[i] * dt
    
    return theta_sim, omega_sim

In [379]:
def train_model(model, t1, theta1, t2, theta2, epochs=500001, lr=0.00001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Преобразуем данные в тензоры
    t1_tensor = torch.tensor(t1, dtype=torch.float32)
    theta1_tensor = torch.tensor(theta1, dtype=torch.float32)
    omega1_tensor = torch.tensor(derivative(t1, theta1), dtype=torch.float32)
    
    t2_tensor = torch.tensor(t2, dtype=torch.float32)
    theta2_tensor = torch.tensor(theta2, dtype=torch.float32)
    omega2_tensor = torch.tensor(derivative(t2, theta2), dtype=torch.float32)
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # 1. Прямой проход (без силы, beta = beta_1)
        theta1_sim, omega1_sim = simulate_trajectory(
            model, t1, theta1[0], omega1_tensor[0].item(), 
            apply_force=False  # beta = beta_1
        )
        loss_theta1 = torch.mean((torch.tensor(theta1_sim) - theta1_tensor)**2)
        
        # 2. Обратный проход (с силой, beta = beta_2)
        ''' theta2_sim, omega2_sim = simulate_trajectory(
            model, t2, theta2[0], omega2_tensor[0].item(), 
            apply_force=True  # beta = beta_2
        )'''
        theta2_sim, omega2_sim = simulate_trajectory(
            model, t2[::-1], theta2[-1], -omega2_tensor[-1].item(), 
            apply_force=True  # beta = beta_2
        )
        loss_theta2 = torch.mean((torch.tensor(theta2_sim[::-1].copy()) - theta2_tensor)**2)
        
        # 3. Loss по ускорениям (опционально)
        a1_pred = model(theta1_tensor[:-1], omega1_tensor[:-1], add_force=False)
        a1_true = (omega1_tensor[1:] - omega1_tensor[:-1]) / (t1_tensor[1:] - t1_tensor[:-1])
        loss_a1 = torch.mean((a1_pred - a1_true)**2)
        
        a2_pred = model(theta2_tensor[:-1], omega2_tensor[:-1], add_force=True)
        a2_true = (omega2_tensor[:-1] - omega2_tensor[1:]) / (t2_tensor[:-1] - t2_tensor[1:])
        loss_a2 = torch.mean((a2_pred - a2_true)**2)
        
        # Общий loss
        loss = (loss_theta1 + 100 * loss_theta2) + 1 * (loss_a1 + 100 * loss_a2)  # Вес для ускорений можно настроить
        loss.backward()
        optimizer.step()
        
        if epoch % 50000 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            # Визуализация
            plt.figure(figsize=(12, 6))
            plt.plot(t1, theta1, 'b-', label='True θ (forward)', alpha=0.7)
            plt.plot(t2, theta2, 'g-', label='True θ (backward)', alpha=0.7)
            plt.plot(t1, theta1_sim, 'r--', label='Pred θ (forward)', alpha=0.7)
            plt.plot(t2, theta2_sim[::-1], 'm--', label='Pred θ (backward)', alpha=0.7)
            plt.legend()
            plt.grid(True)
            plt.show()
    
    return model

In [256]:
def predict_next_zero(model, t_last, theta_last, omega_last, beta, dt=0.01, max_steps=3000):
    """Predict when theta will next cross zero"""
    theta = theta_last
    omega = omega_last
    alpha = model.alpha.item()
    for i in range(max_steps):
        t = t_last + i * dt
        # Euler integration
        acceleration = -alpha * omega - beta * math.sin(theta)
        omega += acceleration * dt
        theta += omega * dt
        
        if i > 0 and theta * prev_theta <= 0:  # Zero crossing detected
            # Linear interpolation for more precise estimate
            t_zero = t - dt + (-prev_theta) * dt / (theta - prev_theta)
            return t_zero
        
        prev_theta = theta
    return t_last + max_steps * dt  # Return max time if no zero crossing found

In [321]:
def simulate_pendulum(model, t_points, theta_init, omega_init, t_force):
    """Simulate pendulum trajectory using current model parameters"""
    theta_sim = np.zeros_like(t_points)
    omega_sim = np.zeros_like(t_points)
    
    theta_sim[0] = theta_init
    omega_sim[0] = omega_init
    
    alpha = model.alpha.item()
    beta_1 = model.beta_1.item()
    beta_2 = model.beta_2.item()
    
    for i in range(1, len(t_points)):
        dt = t_points[i] - t_points[i-1]
        beta = beta_1 if t_force else beta_2
        acceleration = -alpha * omega_sim[i-1] - beta * math.sin(theta_sim[i-1])
        omega_sim[i] = omega_sim[i-1] + acceleration * dt
        theta_sim[i] = theta_sim[i-1] + omega_sim[i] * dt
    
    return theta_sim

In [277]:
def plot_results(t_full, theta_actual, theta_pred1, theta_pred2):
    """Plot actual vs predicted theta values"""
    plt.figure(figsize=(12, 6))
    # Plot actual data
    plt.plot(t_full, theta_actual, 'b-', label='real', alpha=0.7)
    
    # Plot predicted/simulated data
    plt.plot(np.linspace(t.min(), t.max(), 734), theta_pred1, 'r--', label='pred', alpha=0.7)
    plt.plot(np.linspace(t.min(), t.max(), 734), theta_pred2, 'y--', label='pred', alpha=0.7)
    # Mark force application time
    
    plt.legend()
    plt.grid(True)
    plt.show()

tfput обучаемый параметр где мы строим график по всем точкам включая разрыв и как лосс меряем разницу между точками графика.

In [380]:
'''t,theta = load_data(csv_path)
t1,theta1,t2,theta2 = split_data(t,theta)
model = SinglePendulum()
model = train_model(model, t1, theta1, t2, theta2)'''

In [381]:
'''alpha = model.alpha.item()
beta_1 = model.beta_1.item()
beta_2 = model.beta_2.item()
m = 1.0  
g = 10 
l = round(g / beta_1)
miu = alpha * m  
F = beta_2 * l - g 
t_Fput = 5.5
omega_last = derivative(t2[-2:], theta2[-2:])[-1]
t_nextzerotheta = predict_next_zero(model, t2[-1], theta2[-1], omega_last, beta_2)
data = {
            'l': round(l),
            'miu': miu,
            'F': F,
            't_nextzerotheta': t_nextzerotheta,
            't_Fput': t_Fput
}'''

In [378]:
'''theta_pred1 = simulate_pendulum(model, np.linspace(t.min(), t.max(), 734), theta[0], derivative(t[:2], theta[:2])[1], 1)

theta2_sim, _ = simulate_trajectory(
            model, np.linspace(t.min(), t.max(), 734)[::-1], theta[-1], -derivative(t[-2:], theta[-2:])[1], 
            apply_force=True  # beta = beta_2
        )
plot_results(t, theta, theta_pred1, theta2_sim[::-1])'''

In [383]:
def calculate_parameters(csv_path,submission_path): 
    t,theta = load_data(csv_path)
    t1,theta1,t2,theta2 = split_data(t,theta)
    model = SinglePendulum()
    model = train_model(model, t1, theta1, t2, theta2)
    alpha = model.alpha.item()
    beta_1 = model.beta_1.item()
    beta_2 = model.beta_2.item()
    m = 1.0  
    g = 10 
    l = g / beta_1  
    miu = alpha * m  
    F = l * beta_2 - g
    t_Fput = 5.5
    omega_last = derivative(t2[-2:], theta2[-2:])[-1]
    t_nextzerotheta = predict_next_zero(model, t2[-1], theta2[-1], omega_last, beta_2)
    data = {
            'l': round(l),
            'miu': miu,
            'F': round(F),
            't_nextzerotheta': t_nextzerotheta,
            't_Fput': t_Fput
        }
    # Convert to DataFrame.
    df = pd.DataFrame(data,index=[0])
    # Save as a CSV file.
    df.to_csv(submission_path, index=False)
    print(submission_path,"is generated successfully.")
    return 

In [385]:
import zipfile
import os
#Load the training set
TRAIN_PATH = "/bohr/train-08bw/v1/"
calculate_parameters(csv_path = TRAIN_PATH + "pendulum_train.csv", submission_path = "submission_train.csv")

In [93]:
# Load the test set
#“DATA_PATH” is an environment variable for the encrypted test set.
# After submission, the test set can be accessed for system scoring in the following manner, but the participants cannot download it directly.
if os.environ.get('DATA_PATH'):
    DATA_PATH = os.environ.get("DATA_PATH") + "/"  
else:
    print("When the baseline is running, this error message will appear because the test set cannot be read, which is a normal phenomenon.") #When the baseline is running, this error message will appear because the test set cannot be read, which is a normal phenomenon.
calculate_parameters(csv_path = DATA_PATH + "pendulum_testA.csv", submission_path = "submissionA.csv") #Solve the equation for the Public test set （test set A）.
calculate_parameters(csv_path = DATA_PATH + "pendulum_testB.csv", submission_path = "submissionB.csv") #Solve the equation for the Private test set （test set B）.

# Define the files to be packaged and the compressed file name.
files_to_zip = ['submissionA.csv', 'submissionB.csv']
zip_filename = 'submission.zip'

# Create a zip file.
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        # Add files to the zip file. 
        zipf.write(file, os.path.basename(file))

print(f'{zip_filename} is created successfully!')